# LIBRARIES

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors
import seaborn as sns

#Class balance
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTEN
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import ClusterCentroids 
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
sns.set(style='whitegrid',
        rc={'lines.linewidth': 2.5,
        'figure.figsize': (10, 8),
        'text.usetex': False,
        })
%matplotlib inline

# PATH IMAGES

In [ ]:
figures_path = './images' 
if not os.path.exists(figures_path):
    os.makedirs(figures_path) 

# FUNCTIONS

In [ ]:
def load_data_complete_s1(path):
    df_complete=pd.read_csv(path)
    
    # Scenario 1: Tumor_Core & Tumor_Periphery
    # Se procede a eliminar el N_Periphery

    df2 = df_complete.copy()
    df2.drop(df2[df2.classes == "NP"].index, inplace=True)  
    
    # Eliminamos los labels
    features = df2.copy()
    features = features.drop(['classes'], axis=1)
    
    #Extraemos los labels
    labels = df2.copy()
    labels = labels['classes'].values
    
    return features,labels

def load_data_complete_s2(path):
    df_complete=pd.read_csv(path)
    
    # Scenario 2: Normal_Periphery & Tumor_Periphery
    # Se procede a eliminar el T_Core

    df2 = df_complete.copy()
    df2.drop(df2[df2.classes == "TC"].index, inplace=True)  
    
    # Eliminamos los labels
    features = df2.copy()
    features = features.drop(['classes'], axis=1)
    
    #Extraemos los labels
    labels = df2.copy()
    labels = labels['classes'].values
    
    return features,labels

def load_data_complete_s3(path):
    df_complete=pd.read_csv(path)
    # Scenario 3: Tumor_Periphery&Core & Normal_Periphery
    
    # Se procede aislar al N_Periphery
    df2 = df_complete.copy()
    df2.drop(df2[df2.classes == "TP"].index, inplace=True)  
    df2.drop(df2[df2.classes == "TC"].index, inplace=True)  
    
    # Eliminamos el N_Periphery
    df3 = df_complete.copy()
    df3.drop(df3[df3.classes == "NP"].index, inplace=True) 
    
    # y luego se procede a renombrar la columna classes con T_PC, al quedar la unión de estas
    df3["classes"] = "TPC"
    
    # Se procede a crear el DF ya con las clases que corresponde al Escenario 3: Tumor_Periphery&Core & Normal_Periphery
    #df2 N_Periphery
    #df3 T_PC

    df4 = pd.concat([df2,df3]).reset_index(drop=True) 
    
    # Eliminamos los labels
    features = df4.copy()
    features = features.drop(['classes'], axis=1)
    
    #Extraemos los labels
    labels = df4.copy()

    labels = labels['classes'].values
    
    return features,labels

def load_data_complete_s4(path):
    df_complete=pd.read_csv(path)
    
    # Scenario 4 new: Tumor_Core & Tumor_Periphery & N_Periphery

    # Eliminamos los labels
    features = df_complete.copy()
    features = features.drop(['classes'], axis=1)
    
    #Extraemos los labels
    labels = df_complete.copy()
    labels = labels['classes'].values
    
    return features,labels

# LOADING DATA

In [ ]:
X=pd.read_csv("../Data/DATA_Complete_GBM.csv")

In [ ]:
X["classes"].value_counts()

# DATA DISTRIBUTION

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(aspect="equal"))

data = list(X["classes"].value_counts().values)
names = X["classes"].value_counts().index

color_palette_list = ['#C1F0F6','#007ACD',"#EE6055",'#0EBFE9',"#FFD97D","#60D394"]
color_palette_list = ['#0EBFE9',"#60D394","#FFD97D"]

def func(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return "{:.2f}%\n({:d})".format(pct, absolute)

wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),
                                  textprops=dict(color="gray"),colors=color_palette_list[0:],
                                 pctdistance=1.2)

ax.legend(wedges, names,
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1),
         fontsize=15)

plt.setp(autotexts, size=15, weight="bold", color="gray")

plt.subplots_adjust(right=0.7)
plt.savefig(figures_path+"/data_distribution.pdf", bbox_inches = "tight") 
plt.show()

# DATA BALANCING (TRAINING)

In [ ]:
path = '../Data/DATA_Complete_GBM.csv'

featuress1,labelss1=load_data_complete_s1(path)
featuress2,labelss2=load_data_complete_s2(path)
featuress3,labelss3=load_data_complete_s3(path)
featuress4,labelss4=load_data_complete_s4(path)


In [ ]:
testing_size=0.20

#Scenario 1
print("originals labels unique: ",np.unique(labelss1, return_counts=True)) 
X_trains1, X_tests1, y_trains1, y_tests1 = train_test_split(featuress1, labelss1, 
                                                    test_size=testing_size, random_state=21, stratify=labelss1)
sampler = ADASYN(random_state=21,n_jobs=-1) 
print("y_train ORIGINAL labels unique:   ",np.unique(y_trains1, return_counts=True))
print("y_test ORIGINAL labels unique:   ",np.unique(y_tests1, return_counts=True))
X_trains1b, y_trains1b = sampler.fit_resample(X_trains1, y_trains1)             
print("y_train labels unique:   ",np.unique(y_trains1b, return_counts=True))
print("y_test labels unique:    ",np.unique(y_tests1, return_counts=True)) 

#Scenario 2
print("originals labels unique: ",np.unique(labelss2, return_counts=True)) 
X_trains2, X_tests2, y_trains2, y_tests2 = train_test_split(featuress2, labelss2, 
                                                    test_size=testing_size, random_state=21, stratify=labelss2)
sampler = SVMSMOTE(random_state=8,n_jobs=-1) 
print("y_train ORIGINAL labels unique:   ",np.unique(y_trains2, return_counts=True))
print("y_test ORIGINAL labels unique:   ",np.unique(y_tests2, return_counts=True))
X_trains2b, y_trains2b = sampler.fit_resample(X_trains2, y_trains2)             
print("y_train labels unique:   ",np.unique(y_trains2b, return_counts=True))
print("y_test labels unique:    ",np.unique(y_tests2, return_counts=True)) 

#Scenario 3
print("originals labels unique: ",np.unique(labelss3, return_counts=True)) 
X_trains3, X_tests3, y_trains3, y_tests3 = train_test_split(featuress3, labelss3, 
                                                    test_size=testing_size, random_state=21, stratify=labelss3)
sampler = SMOTE(random_state=21,n_jobs=-1) 
print("y_train ORIGINAL labels unique:   ",np.unique(y_trains3, return_counts=True))
print("y_test ORIGINAL labels unique:   ",np.unique(y_tests3, return_counts=True))
X_trains3b, y_trains3b = sampler.fit_resample(X_trains3, y_trains3)             
print("y_train labels unique:   ",np.unique(y_trains3b, return_counts=True))
print("y_test labels unique:    ",np.unique(y_tests3, return_counts=True)) 

#Scenario 4
print("originals labels unique: ",np.unique(labelss4, return_counts=True)) 
X_trains4, X_tests4, y_trains4, y_tests4 = train_test_split(featuress4, labelss4, 
                                                    test_size=testing_size, random_state=21, stratify=labelss4)
sampler = RandomOverSampler(random_state=21) 
print("y_train ORIGINAL labels unique:   ",np.unique(y_trains4, return_counts=True))
print("y_test ORIGINAL labels unique:   ",np.unique(y_tests4, return_counts=True))
X_trains4b, y_trains4b = sampler.fit_resample(X_trains4, y_trains4)             
print("y_train labels unique:   ",np.unique(y_trains4b, return_counts=True))
print("y_test labels unique:    ",np.unique(y_tests4, return_counts=True)) 

# FIGURE OF TRAINING AND TESTING DATA

In [ ]:
nrows=2
ncols=4

fig = plt.figure(figsize=(5*ncols,5*nrows))
fig.subplots_adjust(hspace=0.45, wspace=0.2)

###############################################SCENARIO 1 - Training
i=1
ax = fig.add_subplot(nrows, ncols, i)

dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss1)),"Count":list(np.unique(y_trains1b, return_counts=True)[1])}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[0],size=20)
dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss1)),"Count":list(np.unique(y_trains1, return_counts=True)[1])}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="A"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 2 - Training
i=2
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss2)),"Count":list(np.unique(y_trains2b, return_counts=True)[1])}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[0],size=20)
dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss2)),"Count":list(np.unique(y_trains2, return_counts=True)[1])}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="B"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 3 - Training
i=3
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss3)),"Count":list(np.unique(y_trains3b, return_counts=True)[1])}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[0],size=20)
dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss3)),"Count":list(np.unique(y_trains3, return_counts=True)[1])}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="C"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 4 - Training
i=4
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss4)),"Count":list(np.unique(y_trains4b, return_counts=True)[1])}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[0],size=20)
dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss4)),"Count":list(np.unique(y_trains4, return_counts=True)[1])}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="D"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 1 - Testing
i=5
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss1)),"Count":list(np.unique(y_tests1, return_counts=True))[1]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="gold")
ax.bar_label(ax.containers[0],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="E"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 2 - Testing
i=6
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss2)),"Count":list(np.unique(y_tests2, return_counts=True))[1]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="gold")
ax.bar_label(ax.containers[0],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="F"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 3 - Testing
i=7
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss3)),"Count":list(np.unique(y_tests3, return_counts=True))[1]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="gold")
ax.bar_label(ax.containers[0],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="G"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 4 - Testing
i=8
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss4)),"Count":list(np.unique(y_tests4, return_counts=True))[1]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="gold")
ax.bar_label(ax.containers[0],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="H"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################
model_name = "Data_Balanced_GBM"
fig.savefig("./images"+"/"+model_name+"_CM"+".pdf", bbox_inches = "tight", format='pdf') 

# FIGURE OF TRAINING AND TESTING AND CROSS VALIDATION DATA

In [ ]:
nrows=3
ncols=4

fig = plt.figure(figsize=(5*ncols,5*nrows))
fig.subplots_adjust(hspace=0.45, wspace=0.2)

###############################################SCENARIO 1 - Training
i=1
ax = fig.add_subplot(nrows, ncols, i)

dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss1)),"Count":[309,312]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[0],size=20)
dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss1)),"Count":[309,46]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="A"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 2 - Training
i=2
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss2)),"Count":[206, 206]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[0],size=20)
dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss2)),"Count":[206,  46]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="B"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 3 - Training
i=3
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss3)),"Count":[355, 355]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[0],size=20)
dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss3)),"Count":[206, 355]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="C"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 4 - Training
i=4
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss4)),"Count":[309, 309, 309]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[0],size=20)
dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss4)),"Count":[206, 309,  46]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="D"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 1 - Testing
i=5
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss1)),"Count":[77, 12]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="gold")
ax.bar_label(ax.containers[0],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="E"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 2 - Testing
i=6
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss2)),"Count":[52, 12]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="gold")
ax.bar_label(ax.containers[0],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="F"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 3 - Testing
i=7
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss3)),"Count":[52, 89]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="gold")
ax.bar_label(ax.containers[0],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="G"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 4 - Testing
i=8
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss4)),"Count":[52, 77, 12]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="gold")
ax.bar_label(ax.containers[0],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="H"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################
###############################################SCENARIO 1 - Testing
i=9
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss1)),"Count":[386, 58]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[0],size=20)
dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss1)),"Count":[58, 58]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="I"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 2 - Testing
i=10
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss2)),"Count":[258, 58]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[0],size=20)
dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss2)),"Count":[58, 58]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="J"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 3 - Testing
i=11
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss3)),"Count":[258, 444]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[0],size=20)
dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss3)),"Count":[258, 258]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="K"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################SCENARIO 4 - Testing
i=12
ax = fig.add_subplot(nrows, ncols, i, sharey=ax)

dataFrame_unbalanced = pd.DataFrame(data={"Name":list(np.unique(labelss4)),"Count":[258, 386,  58]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_unbalanced, color="lightseagreen")
ax.bar_label(ax.containers[0],size=20)
dataFrame_balanced   = pd.DataFrame(data={"Name":list(np.unique(labelss4)),"Count":[58, 58, 58]}); 
ax = sns.barplot(x="Name", y="Count", data=dataFrame_balanced, color='lightcyan')
ax.bar_label(ax.containers[1],size=20)

ax.xaxis.set_tick_params(labelsize=25)
ax.set_yticklabels([])
ax.grid(False)
for spine in plt.gca().spines.values():
    spine.set_visible(False)

title="L"
ax.set(xlabel=None, ylabel=None)
ax.set_title(title,fontsize=45,y=1.05)
###############################################
model_name = "Data_Balanced_GBM"
fig.savefig("./images"+"/"+model_name+"_CM"+".pdf", bbox_inches = "tight", format='pdf') 